In [ ]:
import requests
import os
from bs4 import BeautifulSoup as bSoup
from pymongo import MongoClient
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
base_url = "https://warhammer40k.fandom.com"
total_urls = []

In [ ]:
## Pickle Data Loading
import pickle
with open("test", "wb") as fp:   #Pickling
  pickle.dump(docs, fp)
with open("test", "rb") as fp:   # Unpickling
  docs = pickle.load(fp)
docs

Defining functions

In [ ]:
def add_to_total(s):
    if("Category:" not in s and s not in total_urls and "?action=history" not in s):
        for t in total_urls:
            if(t.lower() == s.lower()):
                return

        total_urls.append(s)

def gather_links(categories):
    for s in categories:
        if("Category" not in s):
            res = requests.get(base_url + s)
            soup = bSoup(res.content, 'html.parser')

            add_to_total(s)

            products = soup.find_all("a")
            for e in products:
                if(e == None):
                    continue

                h = e.get("href")
                if(h == None):
                    continue

                if(len(h.split(':')) > 1):
                    if(len(h.split(':')[1]) == 1):
                        continue

                if("/wiki" not in h[0:5]):
                    continue    

                if("/wiki/" in h and "file" not in h and "https" not in h and "edit" not in h and "Special:Search" not in h and "File" not in h):
                    add_to_total(h)
        else:
            for f in ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z","¡"]:
                res = requests.get(base_url + s+"?from="+f)
                soup = bSoup(res.content, 'html.parser')

                products = soup.find_all("a")
                for e in products:
                    if(e == None):
                        continue

                    h = e.get("href")
                    if(h == None):
                        continue

                    if(len(h.split(':')) > 1):
                        if(len(h.split(':')[1]) == 1):
                            continue

                    if("/wiki" not in h[0:5]):
                        continue    

                    if("/wiki/" in h and "file" not in h and "https" not in h and "edit" not in h and "Special:Search" not in h and "File" not in h):
                        add_to_total(h)

Imperium of Man

In [ ]:
imperial_categories = [
    "/wiki/Age_of_the_Imperium",
    "/wiki/Space Marines",
    "/wiki/Astra Militarum",
    "/wiki/Navis_Imperialis",
    "/wiki/Adeptus Mechanicus",
    "/wiki/Adepta Sororitas",
    "/wiki/Planets",
    "/wiki/Imperial Dating System",
    "/wiki/Timeline",
    "/wiki/Great Crusade",
    "/wiki/Horus Heresy",
    "/wiki/Era Indomitus",
    "/wiki/Inquisition",
    "/wiki/Category:History",
    "/wiki/Emperor of Mankind",
    "/wiki/Primarchs",
    "/wiki/Space Marine Legions",
    "/wiki/Codex Astartes",
    "/wiki/Deathwatch",
    "/wiki/List of Space Marine Chapters",
    "/wiki/Category:Space_Marines",
    "/wiki/Commissars",
    "/wiki/Militarum Tempestus",
    "/wiki/Storm Troopers",
    "/wiki/Abhumans",
    "/wiki/Category:Imperial_Guard",
    "/wiki/Aeronautica Imperialis",
    "/wiki/Rogue Traders",
    "/wiki/Category:Imperial_Navy",
    "/wiki/Cult Mechanicus",
    "/wiki/Tech-priest",
    "/wiki/Adeptus Titanicus",
    "/wiki/Imperial Knights",
    "/wiki/Skitarii",
    "/wiki/Forge Worlds",
    "/wiki/Knight Worlds",
    "/wiki/Category:Adepta_Sororitas",
    "/wiki/Terra",
    "/wiki/Mars",
    "/wiki/Cadia",
    "/wiki/Macragge",
    "/wiki/Armageddon",
    "/wiki/Fenris",
    "/wiki/Category:Planets",
]

gather_links(imperial_categories)

Forces of Chaos

In [ ]:
chaos_categories = [
    "/wiki/Chaos",
    "/wiki/Chaos Daemons",
    "/wiki/Forces of Chaos",
    "/wiki/Chaos Gods",
    "/wiki/Chaos Undivided",
    "/wiki/Khorne",
    "/wiki/Tzeentch",
    "/wiki/Nurgle",
    "/wiki/Slaanesh",
    "/wiki/Malice",
    "/wiki/Greater Daemons",
    "/wiki/Lesser Daemons",
    "/wiki/Daemon Prince",
    "/wiki/Eye of Terror",
    "/wiki/Great Rift",
    "/wiki/Chaos Space Marines",
    "/wiki/List of Chaos Space Marine Warbands",
    "/wiki/The Lost and the Damned",
    "/wiki/Dark Mechanicum",
    "/wiki/Chaos Knights",
    "/wiki/Chaos Cults",
    "/wiki/Category:Chaos",
]

gather_links(chaos_categories)

Xenos

In [ ]:
xenos_categories = [
    "/wiki/Craftworld Aeldari",
    "/wiki/Drukhari",
    "/wiki/Leagues of Votann",
    "/wiki/Orks",
    "/wiki/Necrons",
    "/wiki/Tyranids",
    "/wiki/T'au Empire",
    "/wiki/Craftworlds",
    "/wiki/Asuryani Path",
    "/wiki/Aeldari Mythology",
    "/wiki/Category:Eldar",
    "/wiki/Kabal",
    "/wiki/Category:Dark_Eldar",
    "/wiki/Category:Leagues_of_Votann",
    "/wiki/WAAAGH!",
    "/wiki/Ork Empire",
    "/wiki/Category:Ork",
    "/wiki/War_in_Heaven_(Necron)",
    "/wiki/Tomb World",
    "/wiki/Category:Necron",
    "/wiki/Hive_Fleet",
    "/wiki/Category:Tyranid",
    "/wiki/Category:Tau",
]

gather_links(xenos_categories)

In [ ]:
total_urls.sort()
print("Total url count:",len(total_urls))
for s in total_urls:
    print(s)

Gather text and upload to database

In [ ]:
#If the progress stops mid way then you can change the start index to where you last stopped.
start_index = 0
text = []
docs = []


for i in range(start_index, len(total_urls)):
    print("Gathering from: (" + str(i) + ") " + base_url + total_urls[i])
    res = requests.get(base_url + total_urls[i])
    print("Web response code: " + str(res.status_code))

    _text = ""

    soup = bSoup(res.content)
    title = soup.title.text.split("|")[0]
    print("Title: " + title)
    paragraphs = soup.find_all("p")
    print("Found "+ str(len(paragraphs)) + " text elements")
    for p in paragraphs:
        _text += p.text

    docs.append({"title": title, "url": total_urls[i], "content": _text})

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 1500, chunk_overlap = 300)
docs2 = []
load_dotenv("config.env")
uri = os.environ['MONGO_URI']
cluster = MongoClient(uri)
db = cluster["cluster0"]
collection = db["40klore"]

def  get_embeddings(text):
    key = os.environ['OPENAI_KEY']
    client = OpenAI(api_key=key)
    return client.embeddings.create(input = [text], model="text-embedding-3-small").data[0].embedding

start_index =20
run_count = 100
for i in range(start_index, start_index + run_count):
    if(i < 0 or i >= len(docs)):
         continue

    _doc = docs[i]
    print(_doc["title"])
    splits = splitter.split_text(_doc["content"])

    for j, split in enumerate(splits):
        document = {
            "title": _doc["title"] + " " + str(j),
            "url": _doc["url"],
            "content": split,
            "chunk_id": j,
            "embedding": get_embeddings(split)
            }
        docs2.append(
            document
        )
        #print(document)
        #collection.insert_one(document)


In [ ]:
for doc in docs2:
    print(doc["title"])
    print(doc["chunk_id"])
    print(doc["url"])
    print(doc["embedding"][0:100])